<a href="https://colab.research.google.com/github/tanaymukherjee/Deep-Learning-with-PyTorch/blob/master/05_Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Convolutional Neural Network

## Import packages and libraries

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## Device configuration

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Hyper-parameters

In [3]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

## MNIST dataset

In [4]:
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

Extracting ../../data/MNIST/raw/train-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../../data/MNIST/raw


Extracting ../../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../../data/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


## Data loader

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

## Establishing Convolutional Neural Network

In [6]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [7]:
model = ConvNet(num_classes).to(device)

## Loss and optimizer

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Train the model

In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1999
Epoch [1/5], Step [200/600], Loss: 0.0451
Epoch [1/5], Step [300/600], Loss: 0.0731
Epoch [1/5], Step [400/600], Loss: 0.0218
Epoch [1/5], Step [500/600], Loss: 0.1147
Epoch [1/5], Step [600/600], Loss: 0.0422
Epoch [2/5], Step [100/600], Loss: 0.0304
Epoch [2/5], Step [200/600], Loss: 0.0893
Epoch [2/5], Step [300/600], Loss: 0.0725
Epoch [2/5], Step [400/600], Loss: 0.0125
Epoch [2/5], Step [500/600], Loss: 0.0480
Epoch [2/5], Step [600/600], Loss: 0.0218
Epoch [3/5], Step [100/600], Loss: 0.0101
Epoch [3/5], Step [200/600], Loss: 0.0513
Epoch [3/5], Step [300/600], Loss: 0.0331
Epoch [3/5], Step [400/600], Loss: 0.0273
Epoch [3/5], Step [500/600], Loss: 0.0454
Epoch [3/5], Step [600/600], Loss: 0.0156
Epoch [4/5], Step [100/600], Loss: 0.0559
Epoch [4/5], Step [200/600], Loss: 0.0410
Epoch [4/5], Step [300/600], Loss: 0.0274
Epoch [4/5], Step [400/600], Loss: 0.0403
Epoch [4/5], Step [500/600], Loss: 0.0518
Epoch [4/5], Step [600/600], Loss:

## Test the model

In [10]:
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 10000 test images: 99.04 %


## Save the model checkpoint

In [11]:
torch.save(model.state_dict(), 'model.ckpt')